In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import pydicom
from tqdm import tqdm
from tensorflow.keras.utils import to_categorical

In [ ]:
path = "../input/vinbigdata-chest-xray-abnormalities-detection/train.csv"
df = pd.read_csv(path)
df.head(10)

In [ ]:
len(df)

In [ ]:
#path_1 = "../input/vinbigdata-chest-xray-abnormalities-detection/train/47ed17dcb2cbeec15182ed335a8b5a9e.dicom"
#dicom = pydicom.read_file(path_1)

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    """ Convert dicom file to numpy array 
    
    Args:
        path (str): Path to the dicom file to be converted
        voi_lut (bool): Whether or not VOI LUT is available
        fix_monochrome (bool): Whether or not to apply monochrome fix
        
    Returns:
        Numpy array of the respective dicom file 
        
    """
    dicom = pydicom.read_file(path)      # Use the pydicom library to read the dicom file
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":#MONOCHROME1
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.float32)#uint8
        
    return data

In [ ]:
root = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/'
ext = '.dicom'

data = read_xray(root + df.image_id[64] + ext)
plt.imshow(data, 'gray');

In [ ]:
data.shape

In [ ]:
data = df
k = np.random.randint(0,len(data))  #selecting random integer for plotting XRAY


In [ ]:
img_id = data.image_id[k]
img_id

In [ ]:
class_name = data.class_name[k]
class_name

In [ ]:
img_id+ext

In [ ]:
img_path = os.path.join(root, img_id+ext)
#dicom_pixel = read_xray(img_path)

In [ ]:
def show_xray(data, root=root, ext=ext):
    fig, axs = plt.subplots(3,3, figsize=(16,18))
    
    for i in range(9):
        k = np.random.randint(0,len(data))  #selecting random integer for plotting XRAY
        img_id = data.image_id[k]
        class_name = data.class_name[k]
        
        img_path = os.path.join(root, img_id+ext)
        dicom_pixel = read_xray(img_path)
        
        axs[i//3,i%3].imshow(dicom_pixel, cmap='gray')
        axs[i//3,i%3].title.set_text(class_name)
    plt.show()

#show_xray(df)

0 - Aortic enlargement
1 - Atelectasis
2 - Calcification
3 - Cardiomegaly
4 - Consolidation
5 - ILD
6 - Infiltration
7 - Lung Opacity
8 - Nodule/Mass
9 - Other lesion
10 - Pleural effusion
11 - Pleural thickening
12 - Pneumothorax
13 - Pulmonary fibrosis
14 - No finding

In [ ]:
#used label encoder
code = {'Aortic enlargement':0 ,'Atelectasis':1,'Calcification':2,'Cardiomegaly':3,'Consolidation':4,
        'ILD':5,'Infiltration':6,'Lung Opacity':7,'Nodule/Mass':8,
        'Other lesion':9,'Pleural effusion':10,'Pleural thickening':11,'Pneumothorax':12,'Pulmonary fibrosis':13,'No finding':14}

def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x 

In [ ]:
import glob
root = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/'
img_path = glob.glob('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/*.*')
len(img_path)

In [ ]:
dicom_pixel = read_xray(img_path[5620])

In [ ]:
plt.imshow(dicom_pixel, 'gray')

In [ ]:
img_path[500]

In [ ]:
images_batch = np.random.choice(df.index, size=16)
images_batch

In [ ]:
data = df.index
print(data[15010])

In [ ]:
path = "../input/vinbigdata-chest-xray-abnormalities-detection/train.csv"
df_1 = pd.read_csv(path)
root = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/'
ext = '.dicom'
data = df_1
X_train = []
y_train = []
i=0
s=128
for file in tqdm(data.index): 
    img_id = data.image_id[file]#take image ID
    #class_name = data.class_name[file]
    idx = data.class_id[file]#take image class ID (0 1 .....14)
    
    img_path = os.path.join(root, img_id+ext)
    dicom_pixel = read_xray(img_path)
    
    image_array = cv2.resize(dicom_pixel , (s,s), interpolation = cv2.INTER_AREA)
    
    X_train.append(list(image_array))
    y_train.append(idx)#code[class_name]
    if (file == 1):
        break
'''    else:
        i=i+1'''

In [ ]:
# define the shape of low resolution image (LR) for resize
path = "../input/vinbigdata-chest-xray-abnormalities-detection/train.csv"
df_1 = pd.read_csv(path)
data = df_1
root = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/'
ext = '.dicom'
image_shape = (224, 224)
img_path = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train/*.*'
def sample_images(batch_size, image_shape):
    
    # create the list of all images, which are inside of data_dir catalogue
    #all_images = glob.glob(data_dir)
    
    # select a random batch with images
    images_batch = np.random.choice(15000, size=batch_size)#total_images

    X_train = []
    y_train = []

    for img in images_batch:
        # take the numpy ndarray from the current image
        img_id = data.image_id[img]
        idx = data.class_id[img]#take image class ID (0 1 .....14)
        img_path = os.path.join(root, img_id+ext)
        dicom_pixel = read_xray(img_path)
        image_array = cv2.resize(dicom_pixel , image_shape, interpolation = cv2.INTER_AREA)
        X_train.append(list(image_array))
        y_train.append(idx)#code[class_name]
        
    X_train=np.array(X_train)
    X_train = X_train.reshape(-1,224,224,1)
    y_train=to_categorical(y_train)
    # convert lists into numpy ndarrays
    return X_train, y_train

In [ ]:
len(images_batch)

In [ ]:
batch_size = 8
(X_train,y_train)= sample_images(batch_size, image_shape)

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPool2D,BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD,Adam,RMSprop,Adagrad
from keras.regularizers import l1
import numpy as np
np.random.seed(512)

s = 224
model = Sequential()
#model.add(BatchNormalization(input_shape=(s,s,1)))
model.add(Conv2D(16, kernel_size=(3,3), padding='same', activation='relu',input_shape=(s,s,1)))
model.add(MaxPool2D(pool_size=(2,2)))
#,activity_regularizer=l1(0.000001)

model.add(Conv2D(32, kernel_size=(3,3),padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3),padding='same', activation='relu'))#
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3),padding='same', activation='relu')) 
model.add(Conv2D(64, kernel_size=(3,3),padding='same', activation='relu'))

model.add(Conv2D(256, kernel_size=(3,3),padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3),padding='same', activation='relu'))#64,101
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.08))

model.add(Flatten())
model.add(Dense(2024, activation='relu'))
model.add(Dropout(0.13))

model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.13))

model.add(Dense(4096, activation='relu'))
#model.add(Dropout(0.11))
model.add(Dense(2024, activation='relu'))#1024
#model.add(Dropout(0.11))
model.add(Dense(15, activation='softmax'))



#=========================================================================================

model.summary()

opt = Adam(learning_rate=0.00001)
# Compile the model
model.compile(loss=tensorflow.keras.losses.categorical_crossentropy, optimizer=opt, metrics=["categorical_accuracy"]) #categorical_accuracy
# plot model architecture
plot_model(model, show_shapes=True, to_file='Covid-19_Model.png')

In [ ]:
#del model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
#ModelCheckpoint means save best weights
model_chkpt = ModelCheckpoint('best_mod.h5', save_best_only=True, monitor='accuracy')
learning_rate_reduction1 = ReduceLROnPlateau(monitor='val_loss', 
                                             patience=2, verbose=1, factor=0.5,mode="min", min_lr=0.0000001)
learning_rate_reduction2 = ReduceLROnPlateau(monitor='loss', 
                                             patience=2, verbose=1, factor=0.5,mode="min", min_lr=0.000001)

early_stopping = EarlyStopping(monitor='val_categorical_accuracy', restore_best_weights=False, patience=20)#val_loss,val_categorical_accuracy
callbacks1=[early_stopping,learning_rate_reduction2,learning_rate_reduction1]

In [ ]:
#del history

In [ ]:
#(X_train,y_train)= sample_images(batch_size, image_shape)

In [ ]:
FAST_RUN=True
#if FAST_RUN else 6
batch_size1=128
batch_size2=8
batch_size3=64
epochs1=32
for epoch in range(epochs1):
    print(epoch)
    (X_train,y_train)= sample_images(batch_size1, image_shape)
    (X_test,y_test)= sample_images(batch_size3, image_shape)
    history = model.fit(X_train, y_train ,epochs=2,validation_data=(X_test, y_test),
                        batch_size=batch_size2, shuffle=True, callbacks=callbacks1)#, verbose=1
    print("========================================================================================")
    
#validation_data=(X_test, y_test),validation_split=0.20,

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12, 3))
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['categorical_accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_categorical_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)